In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

2025-08-18 17:34:15.197014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 17:34:15.931759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-18 17:34:15.931858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-18 17:34:15.935311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 17:34:16.344016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 17:34:16.347504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copying weights into the model

In [3]:
model.compile()

In [4]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Test with background data

In [16]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:]

input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

X = X.reshape(X.shape[0],-1)

In [23]:
from tqdm import tqdm
batch_size = 128
pred_qkeras = []

for start in tqdm(range(0, 1024, batch_size), desc="Qkeras preds"):
        end = min(start + batch_size, 1024)
        
        pred = model.predict(input_quantizer(X[start:end]))
        quantized_pred = output_quantizer(pred)
        pred_qkeras.append(quantized_pred)

Qkeras preds:   0%|          | 0/8 [00:00<?, ?it/s]

4/4 [==============================] - 0s 1ms/step


Qkeras preds:  25%|██▌       | 2/8 [00:00<00:00, 19.61it/s]

4/4 [==============================] - 0s 1ms/step


Qkeras preds:  62%|██████▎   | 5/8 [00:00<00:00, 20.79it/s]

4/4 [==============================] - 0s 2ms/step


Qkeras preds: 100%|██████████| 8/8 [00:00<00:00, 20.90it/s]


### Conversion to a QONNX model

In [20]:
import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)
qonnx_model = qonnx_model.transform(InferShapes())

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
2025-08-18 18:04:14.654024: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-08-18 18:04:14.654208: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-08-18 18:04:14.711346: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-08-18 18:04:14.711493: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [93]:
pred_qonnx = []

for i in tqdm(range(0, 128)):
    idict = {qonnx_model.graph.input[0].name: input_quantizer(X[i]).numpy().reshape(1,57)}
    odict = oxe.execute_onnx(qonnx_model, idict, True)
    pred = output_quantizer(odict[qonnx_model.graph.output[0].name])
    pred_qonnx.append(pred)

100%|██████████| 128/128 [01:16<00:00,  1.67it/s]


In [79]:
print(pred_qkeras[0].numpy().flatten())

[ 4.4375  1.375   1.75    0.8125  4.875   3.75    4.1875  2.25    0.8125
  3.8125  0.9375  0.3125  5.1875  1.      8.4375  1.25    2.125   0.75
  2.125   1.625   2.1875  3.375   2.5     1.5     2.      3.0625  2.375
  1.5625  2.25    2.      1.375   0.6875  1.      0.875   1.      0.8125
  1.625   2.125   2.1875  1.75    1.4375  0.8125  3.625   0.75    1.375
  2.125   4.25    1.75    0.875   1.875   1.6875  3.      3.      1.4375
  1.375   0.9375  1.0625  1.5     2.625   1.5     0.875   3.6875  1.5
  0.6875  2.375   1.3125  4.375   0.875   2.875   2.0625  2.5     0.5
  1.1875  1.25    6.125   0.6875  1.5625  0.875   1.3125  0.6875  1.5625
  1.5     2.625   1.5625  0.5     1.25    2.5     2.0625  0.3125  2.625
  1.5625  1.25    0.9375  2.75    1.875   0.75    1.5     2.5625  0.75
  7.625   6.5     1.6875  1.375   0.9375  4.375   3.25    4.1875  0.5625
  2.9375  2.5     2.25    0.5625  1.9375  2.875   3.25    2.5625  1.0625
  2.3125  0.5625 13.0625  1.125   1.4375  3.75    0.625   0.625 

In [88]:
print(pred_qonnx[0])

[[4.4136963]]


In [94]:
pred_qonnx = numpy.array(pred_qonnx)
print(numpy.hstack((pred_qkeras[0].numpy()[:10], pred_qonnx[:10].reshape(-1,1))))

[[4.4375 4.4375]
 [1.375  1.375 ]
 [1.75   1.75  ]
 [0.8125 0.8125]
 [4.875  4.875 ]
 [3.75   3.75  ]
 [4.1875 4.1875]
 [2.25   2.25  ]
 [0.8125 0.8125]
 [3.8125 3.8125]]
